In [40]:
import os
from PIL import Image

In [5]:
import numpy as np

# Parameters
num_classes = 7
num_samples_per_class = 100
image_size = 224
channels = 3

# Generate dummy image data
training_images = np.random.rand(num_classes * num_samples_per_class, image_size, image_size, channels)
training_labels = np.repeat(np.arange(num_classes), num_samples_per_class)

print("Training images shape:", training_images.shape)
print("Training labels shape:", training_labels.shape)

Training images shape: (700, 224, 224, 3)
Training labels shape: (700,)


In [16]:
batch_size = 32
image_size = 224
base_dir = 'dummy_dataset'
class_folders = sorted(os.listdir(base_dir))

In [18]:
image_paths = []
labels = []
for class_name in class_folders:
    class_dir = os.path.join(base_dir, class_name)
    for fname in os.listdir(class_dir):
        image_paths.append(os.path.join(class_dir, fname))
        labels.append(int(class_name))  

In [45]:
def _load_image(path):
    img = Image.open(path).resize((image_size, image_size))
    img = np.array(img) / 255.0  # Normalize image to [0, 1]
    return img

In [ ]:
def __getitem__(self, idx):
    batch_indices = self.indices[idx * self.batch_size: (idx + 1) * self.batch_size]
    batch_x = []
    for i in batch_indices:
        anchor, positive, negative = self._sample_triplet(i)
        batch_x.append([anchor, positive, negative])
    batch_x = np.array(batch_x)
    return [batch_x[:, 0], batch_x[:, 1], batch_x[:, 2]], None

def _sample_triplet(self, index):
    anchor_class = self.labels[index]
    positive_indices = [i for i, label in enumerate(self.labels) if label == anchor_class and i != index]
    if not positive_indices:
        raise ValueError(f"Class {anchor_class} has only one sample.")
    positive_idx = np.random.choice(positive_indices)
    
    negative_class = np.random.choice(list(set(self.class_folders) - {str(anchor_class)}))
    negative_indices = [i for i, label in enumerate(self.labels) if label == int(negative_class)]
    negative_idx = np.random.choice(negative_indices)
    
    anchor = self._load_image(self.image_paths[index])
    positive = self._load_image(self.image_paths[positive_idx])
    negative = self._load_image(self.image_paths[negative_idx])
    
    return anchor, positive, negative

In [ ]:
def __getitem__(self, idx):
    batch_x = []
    for _ in range(self.batch_size):  # Generate `batch_size` number of triplets
        anchor, positive, negative = self._sample_triplet()
        batch_x.append([anchor, positive, negative])
    batch_x = np.array(batch_x)
    # Return the triplet images and dummy labels
    return [batch_x[:, 0], batch_x[:, 1], batch_x[:, 2]], np.zeros((self.batch_size, 1))

def _sample_triplet(self):
    # Sample a triplet: anchor, positive (same class as anchor), negative (different class)
    anchor_class = np.random.choice(self.class_folders)
    
    # Create a temporary list to exclude the anchor class
    remaining_classes = list(set(self.class_folders) - {anchor_class})
    negative_class = np.random.choice(remaining_classes)
    
    # Sample two distinct indices from the same class for anchor and positive
    anchor_idx, positive_idx = np.random.choice(
        [i for i, label in enumerate(self.labels) if label == int(anchor_class)], 2, replace=False)
    # Sample one index from a different class for negative
    negative_idx = np.random.choice(
        [i for i, label in enumerate(self.labels) if label == int(negative_class)], 1)[0]
    
    anchor = self._load_image(self.image_paths[anchor_idx])
    positive = self._load_image(self.image_paths[positive_idx])
    negative = self._load_image(self.image_paths[negative_idx])
    
    return anchor, positive, negative

In [48]:
anchor_class = np.random.choice(class_folders)
negative_class = np.random.choice( list(set(class_folders) - {anchor_class}))

anchor_idx, positive_idx = np.random.choice(
    [i for i, label in enumerate(labels) if label == int(anchor_class)], 2, replace=False)
negative_idx = np.random.choice(
    [i for i, label in enumerate(labels) if label == int(negative_class)], 1)[0]

anchor = _load_image(image_paths[anchor_idx])
positive = _load_image(image_paths[positive_idx])
negative = _load_image(image_paths[negative_idx])

In [58]:
batch_x = np.array(batch_x)

(3, 3, 224, 224, 3)

In [69]:
batch_x = np.array(batch_x)

In [70]:
batch_x.shape

(5, 3, 224, 224, 3)

In [72]:
batch_x[:, 1].shape

(5, 224, 224, 3)

In [62]:
batch_x = []

In [68]:
batch_x.append([anchor, positive, negative])

In [64]:
class_folders.remove(anchor_class)

In [ ]:
batch_x = []
for _ in range(batch_size):  # Generate `batch_size` number of triplets
    anchor, positive, negative = _sample_triplet()
    batch_x.append([anchor, positive, negative])
batch_x = np.array(batch_x)

In [ ]:
return [batch_x[:, 0], batch_x[:, 1], batch_x[:, 2]], np.zeros((self.batch_size, 1))

In [74]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Lambda, Conv2D, MaxPooling2D, Flatten, Dense, concatenate
import tensorflow.keras.backend as K

inputs = Input(shape=input_shape)
x = Conv2D(32, (3, 3), activation='relu')(inputs)
x = MaxPooling2D((2, 2))(x)
x = Conv2D(64, (3, 3), activation='relu')(x)
x = MaxPooling2D((2, 2))(x)
x = Flatten()(x)
x = Dense(128, activation='relu')(x)
model = Model(inputs, x)

NameError: name 'input_shape' is not defined

In [ ]:
def triplet_loss(y_true, y_pred, alpha=0.2):
    total_length = y_pred.shape.as_list()[-1]
    anchor_length = total_length // 3
    anchor, positive, negative = y_pred[:, 0:anchor_length], y_pred[:, anchor_length:2*anchor_length], y_pred[:, 2*anchor_length:]
    
    pos_dist = K.sum(K.square(anchor - positive), axis=1)
    neg_dist = K.sum(K.square(anchor - negative), axis=1)
    
    basic_loss = pos_dist - neg_dist + alpha
    loss = K.maximum(basic_loss, 0.0)
    
    return loss

In [ ]:
import tensorflow as tf

def triplet_loss(anchor, positive, negative, alpha=0.2):
    # Calculate the positive and negative distances
    pos_dist = tf.reduce_sum(tf.square(anchor - positive), axis=1)
    neg_dist = tf.reduce_sum(tf.square(anchor - negative), axis=1)
    
    # Calculate the triplet loss
    basic_loss = pos_dist - neg_dist + alpha
    loss = tf.maximum(basic_loss, 0.0)
    
    return tf.reduce_mean(loss)
